In [ ]:
"""To modify the `improved_fidelity` static method of entanglement purification protocol (`BBPSSW`).

The modification is aimed at demonstrating entanglement purification under a different error model,
i.e. only single-qubit bit-flip errors exist. Thus the imperfect entangled states are no longer Werner-type
but simply a mixture of two Bell states: `desired_state` and its counterpart under single-qubit bit-flip.
In this case the improved fidelity is different from more general case where Werner states are considered.
"""

from sequence.entanglement_management.purification import BBPSSW


def test_fid(F):
    """Only for testing this approach of modification."""
    
    return 0.88

def original_fid(F):
    """Only for record of original method in script before modification."""
    
    return (F ** 2 + ((1 - F) / 3) ** 2) / (F ** 2 + 2 * F * (1 - F) / 3 + 5 * ((1 - F) / 3) ** 2)

def new_improved_fid(F):
    """The improved fidelity corresponding under bit-flip error model."""
    
    return F ** 2 / (F ** 2 + (1 - F) ** 2)

BBPSSW.improved_fidelity = new_improved_fid

In [ ]:
"""To modify the `_set_state_with_fidelity` function defined in SeQUeNCe BSM modele.

This modification is corresponding to the previous modification of purification improved fidelity.
Previously the physically generated entangled states are of Werner type, to demonstrate the difference
now they should be mixture of only two Bell states as discussed above.
"""

import sequence.components.bsm as bsm_module
from sequence.kernel.quantum_manager import KET_STATE_FORMALISM, DENSITY_MATRIX_FORMALISM


BSM = bsm_module.BSM

def test_method(keys, desired_state, fidelity, rng, qm):
    """Only for testing this approach of modification."""
    
    possible_states = [BSM._phi_plus, BSM._phi_minus,
                       BSM._psi_plus, BSM._psi_minus]
    assert desired_state in possible_states

    if qm.formalism == KET_STATE_FORMALISM:
        qm.set(keys, desired_state)

    elif qm.formalism == DENSITY_MATRIX_FORMALISM:
        state = outer(desired_state, desired_state)
        qm.set(keys, state)

    else:
        raise Exception("Invalid quantum manager with formalism {}".format(qm.formalism))
        
def _set_state_with_fidelity(keys, desired_state, fidelity, rng, qm):
    """Only for record of original method in script before modification."""

    possible_states = [BSM._phi_plus, BSM._phi_minus,
                       BSM._psi_plus, BSM._psi_minus]
    assert desired_state in possible_states

    if qm.formalism == KET_STATE_FORMALISM:
        probabilities = [(1 - fidelity) / 3] * 4
        probabilities[possible_states.index(desired_state)] = fidelity
        state_ind = rng.choice(4, p=probabilities)
        qm.set(keys, possible_states[state_ind])

    elif qm.formalism == DENSITY_MATRIX_FORMALISM:
        multipliers = [(1 - fidelity) / 3] * 4
        multipliers[possible_states.index(desired_state)] = fidelity
        state = zeros((4, 4))
        for mult, pure in zip(multipliers, possible_states):
            state = add(state, mult * outer(pure, pure))
        qm.set(keys, state)

    else:
        raise Exception("Invalid quantum manager with formalism {}".format(qm.formalism))

def new_set_state(keys, desired_state, fidelity, rng, qm):
    """The new function to set generated entangled states under a different assumed error model."""

    possible_states = [BSM._phi_plus, BSM._phi_minus,
                       BSM._psi_plus, BSM._psi_minus]
    assert desired_state in possible_states

    if qm.formalism == KET_STATE_FORMALISM:
        probabilities = [0] * 4
        probabilities[possible_states.index(desired_state)] = fidelity
        if desired_state == BSM._psi_plus:
            probabilities[possible_states.index(BSM._phi_plus)] = 1 - fidelity
        elif desired_state == BSM._psi_minus:
            probabilities[possible_states.index(BSM._phi_minus)] = 1 - fidelity
        state_ind = rng.choice(4, p=probabilities)
        qm.set(keys, possible_states[state_ind])

    elif qm.formalism == DENSITY_MATRIX_FORMALISM:
        multipliers = [0] * 4
        multipliers[possible_states.index(desired_state)] = fidelity
        if desired_state == BSM._psi_plus:
            probabilities[possible_states.index(BSM._phi_plus)] = 1 - fidelity
        elif desired_state == BSM._psi_minus:
            probabilities[possible_states.index(BSM._phi_minus)] = 1 - fidelity
        state = zeros((4, 4))
        for mult, pure in zip(multipliers, possible_states):
            state = add(state, mult * outer(pure, pure))
        qm.set(keys, state)

    else:
        raise Exception("Invalid quantum manager with formalism {}".format(qm.formalism))


bsm_module._set_state_with_fidelity = new_set_state